In [81]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [82]:
df=pd.read_csv('titanic/train.csv')

In [83]:
df.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [84]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [85]:
X_train,X_test,y_train,y_test=train_test_split(df.drop(columns=['Survived']),
                                                       df['Survived'],
                                                       test_size=0.2,
                                               random_state=42)

In [86]:
X_train.head(1)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5,S


In [87]:
#  imputation transformer (Filling the missing Values)
trf1=ColumnTransformer([
        ('impute_age',SimpleImputer(strategy='mean'),[2]),
        ('impute_embarked',SimpleImputer(strategy='most_frequent'),[-1])
    ],remainder='passthrough')

In [88]:
# one hot encoding 
trf2=ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6])
],remainder='passthrough')

In [89]:
# Scaling (Applies MinMaxScaler on all columns (0,10))

trf3=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
     ])

In [90]:
# Feature Selection (seelect the top 5 columns)
trf4=SelectKBest(score_func=chi2,k=8)

In [91]:
# Train the Model
trf5=DecisionTreeClassifier()

# CREATE PIPELINE

In [92]:
pipe=Pipeline([
    ('trf1',trf1),
        ('trf2',trf2),
        ('trf3',trf3),
        ('trf4',trf4),
        ('trf5',trf5)
])

# Pipeline vs Make_pipeline

- pipeline requires naming of steps , make_pipeline does not

- (Similarly Column_Transformer vs make_column_transformer)

In [93]:
# # Alternate method
# pipe=make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [94]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [-1])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001786A342B60>)),
                ('trf5', DecisionTreeClassifier())])

# Explore the Pipeline

In [95]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

In [96]:
# Display PIpeline

from sklearn import set_config
set_config(display='diagram')

In [97]:
# Predict
y_pred=pipe.predict(X_test)

In [98]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

# Cross Validation Using Pipeline

In [99]:
#  cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy').mean()

0.6391214419383433

# GridSearch using Pipeline

In [100]:
# gridsearchcv
params={
    'trf5__max_depth':[1,2,3,4,5,None]
}


In [101]:
from sklearn.model_selection import GridSearchCV
grid=GridSearchCV(pipe,params,cv=5,scoring='accuracy')
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [-1])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x000001786A342B60>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [102]:
grid.best_score_

0.6391214419383433

In [103]:
grid.best_params_

{'trf5__max_depth': 2}

# EXporting the PipeLine

In [104]:
# export 
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))